In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
pd.set_option('float_format', '{:f}'.format)
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Presecription data
RX = pd.read_csv(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\APLD Raw Data\RX_2l.txt.gz",sep="|")


In [3]:
#Get drug information
drug_label = pd.read_csv(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\Drug_table_with_type.csv",sep='|')
drug_label = drug_label[['NDC','type','BGI','DRUG_STRENGTH']]
RX_merged = RX.merge(drug_label,how="left",on="NDC")
RX_merged = RX_merged[['PATIENT_ID','SERVICE_DATE','NDC','type','DSPNSD_QTY','DAYS_SUPPLY','DRUG_STRENGTH','DSPNSD_QTY','REFILL_CODE']]

In [69]:
RX_merged = RX.merge(drug_label,how="left",on="NDC")
RX_merged = RX_merged[['PATIENT_ID','SERVICE_DATE','NDC','DIAGNOSIS_CODE','type','DSPNSD_QTY','DAYS_SUPPLY','DRUG_STRENGTH','DSPNSD_QTY','REFILL_CODE']]

In [70]:
BC_SN = pd.read_excel(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\BC_SN ICD Code.xlsx")

In [71]:
RX_merged = RX_merged.merge(BC_SN,how="left", left_on='DIAGNOSIS_CODE',right_on ='diagnosis_cd').drop(columns=['diagnosis_cd','DIAGNOSIS_CODE','version','sub_indication_cd'])

In [75]:
RX_merged =  RX_merged.dropna(subset=['indication_cd'])

In [78]:
# Dealing with the service date
def lookup(s):
    """ 
    This is a faster approach to convert string to datetime object
    """
    dates = {date:pd.to_datetime(date) for date in s.unique()}
    return s.apply(lambda v: dates[v])


RX_merged['SERVICE_DATE'] = lookup(RX_merged['SERVICE_DATE'])

In [72]:
RX_merged.head()

,PATIENT_ID,SERVICE_DATE,NDC,type,DSPNSD_QTY,DAYS_SUPPLY,DRUG_STRENGTH,DSPNSD_QTY,REFILL_CODE,indication_cd
0,143389772,11/28/2014,93747489,TAM,84.000000,21,500 MG,84.000000,1.000000,BC
1,143389772,11/07/2014,93747489,TAM,84.000000,21,500 MG,84.000000,1.000000,BC
2,143389772,12/18/2014,93747489,TAM,84.000000,21,500 MG,84.000000,1.000000,BC
3,143389772,10/21/2014,93747489,TAM,84.000000,21,500 MG,84.000000,0.000000,BC
4,325994833,09/11/2014,93747489,TAM,84.000000,21,500 MG,84.000000,0.000000,SN


In [76]:
len(RX_merged.indication_cd.unique())

2

## Drug info

In [79]:
min_dates = RX_merged.groupby(['PATIENT_ID'])['SERVICE_DATE'].min()
RX_merged['Earliest_DIAG_Date'] = RX_merged.apply(lambda row: min_dates.loc[row['PATIENT_ID']], axis=1)
RX_merged['Days_Diff'] = RX_merged['SERVICE_DATE'] - RX_merged['Earliest_DIAG_Date']
RX_merged['Days_Diff'] = RX_merged['Days_Diff'].dt.days

In [80]:
drug_type_dummies =pd.get_dummies(RX_merged.type,prefix='drug_type')
drug_type_dummies.reset_index(inplace=True)
RX_merged.reset_index(inplace=True)
RX_merged = RX_merged.merge(drug_type_dummies,how="left",on="index")

In [81]:
RX_merged.head()

,index,PATIENT_ID,SERVICE_DATE,NDC,type,DSPNSD_QTY,DAYS_SUPPLY,DRUG_STRENGTH,DSPNSD_QTY,REFILL_CODE,indication_cd,Earliest_DIAG_Date,Days_Diff,drug_type_AFI,drug_type_AI,drug_type_FAS,drug_type_IBR,drug_type_KIS,drug_type_LET,drug_type_OTHER,drug_type_TAM,drug_type_VER
0,0,143389772,2014-11-28,93747489,TAM,84.000000,21,500 MG,84.000000,1.000000,BC,2014-10-21,38,0,0,0,0,0,0,0,1,0
1,1,143389772,2014-11-07,93747489,TAM,84.000000,21,500 MG,84.000000,1.000000,BC,2014-10-21,17,0,0,0,0,0,0,0,1,0
2,2,143389772,2014-12-18,93747489,TAM,84.000000,21,500 MG,84.000000,1.000000,BC,2014-10-21,58,0,0,0,0,0,0,0,1,0
3,3,143389772,2014-10-21,93747489,TAM,84.000000,21,500 MG,84.000000,0.000000,BC,2014-10-21,0,0,0,0,0,0,0,0,1,0
4,4,325994833,2014-09-11,93747489,TAM,84.000000,21,500 MG,84.000000,0.000000,SN,2014-05-22,112,0,0,0,0,0,0,0,1,0


In [82]:
#Make sure that we only look for those who has only 1 type of drug as the first line and the first line drug is in those 10 brands
first_line = RX_merged[RX_merged['SERVICE_DATE']==RX_merged['Earliest_DIAG_Date']]

first_line_drug_count = pd.DataFrame(first_line.groupby(['PATIENT_ID','Earliest_DIAG_Date'])['type'].nunique())
first_line_drug_count.reset_index(inplace=True)

# List of patient whose first line has more than one type of drug
Patient_more_than_one_drug = first_line_drug_count[first_line_drug_count['type']!=1]['PATIENT_ID'].unique().tolist()

# List of patient who has only one drug as the first line but is not in those 10 brands
Patient_one_drug = first_line[~first_line.PATIENT_ID.isin(Patient_more_than_one_drug)]
Patient_other_drug = Patient_one_drug[Patient_one_drug['type']=='OTHER']['PATIENT_ID'].unique().tolist()

Patient_to_remove = Patient_more_than_one_drug + Patient_other_drug

RX_merged = RX_merged[~RX_merged.PATIENT_ID.isin(Patient_to_remove)]

In [83]:
len(RX_merged.PATIENT_ID.unique())

4908

#### Now RX_merged is cleaned and only contains those whose first line has only one drug which belong to those 10 brands

#### Find the date where the patient has proceed to second line

In [9]:
# Find the first line drug for each patient
first_drug = RX_merged[(RX_merged['SERVICE_DATE'] == RX_merged['Earliest_DIAG_Date'])][['PATIENT_ID','type']]

first_drug = first_drug.rename(columns={'type':'first_drug'})

RX_merged = RX_merged.merge(first_drug,how='left',on='PATIENT_ID')

In [10]:
patient_drug_count = pd.DataFrame(RX_merged.groupby(['PATIENT_ID'])['type'].nunique())

patient_drug_count.reset_index(inplace=True)

patientID_one_drug = patient_drug_count[patient_drug_count.type==1]['PATIENT_ID'].unique().tolist()

patientID_more_than_one_drug = patient_drug_count[patient_drug_count.type>1]['PATIENT_ID'].unique().tolist()

In [11]:
patient_one_drug = RX_merged[RX_merged.PATIENT_ID.isin(patientID_one_drug)][['PATIENT_ID','SERVICE_DATE']]


# Create the label to indicate whether the patient has ever proceed to second line
patient_one_drug['2nd_line'] = 0

# If the drug type of a patient is all the same as the first drug, then these patients have never proceed to second line
# Here we find the latest diag date
patient_one_drug_latest_date = patient_one_drug.groupby(['PATIENT_ID'])['SERVICE_DATE'].max()
patient_one_drug['latest_DIAG_Date'] = patient_one_drug.apply(lambda row: patient_one_drug_latest_date.loc[row['PATIENT_ID']], axis=1)


In [11]:
patient_one_drug.head()

,PATIENT_ID,SERVICE_DATE,2nd_line,latest_DIAG_Date
22,915900636,2019-12-31,0,2019-12-31
35,1167080703,2019-12-05,0,2019-12-05
77,1091256399,2019-10-17,0,2019-10-17
81,1727274390,2019-10-17,0,2019-11-18
234,76521234,2019-11-11,0,2019-12-09


In [12]:
patient_more_than_one_drug =RX_merged[RX_merged.PATIENT_ID.isin(patientID_more_than_one_drug)]
patient_more_than_one_drug = patient_more_than_one_drug[['PATIENT_ID','SERVICE_DATE','type','Days_Diff','first_drug']]

patient_more_than_one_drug['type_diff'] = np.where(patient_more_than_one_drug['type']!=patient_more_than_one_drug['first_drug'],1,0)

In [13]:
earliest_switch_date = patient_more_than_one_drug[(patient_more_than_one_drug['Days_Diff']>30) & (patient_more_than_one_drug['type_diff']==1)]

earliest_switch_date = earliest_switch_date.groupby(['PATIENT_ID'])['SERVICE_DATE'].min()

earliest_switch_date = pd.DataFrame(earliest_switch_date)

earliest_switch_date.reset_index(inplace=True)

earliest_switch_date = earliest_switch_date.rename(columns={'SERVICE_DATE':"earliest_switch_date"})

In [14]:
earliest_switch_date.head()

,PATIENT_ID,earliest_switch_date
0,29425424,2017-11-17
1,29537719,2016-11-20
2,29563713,2018-04-24
3,29594645,2018-09-10
4,29703441,2014-09-23


In [15]:
patient_more_than_one_drug = patient_more_than_one_drug.merge(earliest_switch_date,how="left",on=['PATIENT_ID'])

patient_more_than_one_drug['2nd_line'] = np.where(patient_more_than_one_drug['earliest_switch_date'].isna(),1,2)


In [16]:
# Get the latest diagnosis date for those who has more than one drug but without record over 30 days
PatientID_within_30_days = patient_more_than_one_drug[patient_more_than_one_drug['2nd_line']==1]['PATIENT_ID'].unique().tolist()

Patient_within_30_days = RX_merged[RX_merged.PATIENT_ID.isin(PatientID_within_30_days)]

Patient_within_30_days = Patient_within_30_days[['PATIENT_ID','SERVICE_DATE']]

latest_Date_Patient_within_30_days = Patient_within_30_days.groupby(['PATIENT_ID'])['SERVICE_DATE'].max()
Patient_within_30_days['latest_DIAG_Date'] = Patient_within_30_days.apply(lambda row: latest_Date_Patient_within_30_days.loc[row['PATIENT_ID']], axis=1)

Patient_within_30_days['2nd_line'] = 1

In [17]:
Patient_within_30_days.head()

,PATIENT_ID,SERVICE_DATE,latest_DIAG_Date,2nd_line
27,1732748142,2019-12-27,2019-12-27,1
1060,1239469724,2019-12-24,2019-12-24,1
1375,493424557,2019-12-20,2019-12-20,1
1547,1768004395,2019-12-16,2019-12-16,1
1655,358715626,2019-12-18,2019-12-18,1


In [18]:
patient_switch_therapy = patient_more_than_one_drug[patient_more_than_one_drug['2nd_line']==2][['PATIENT_ID','SERVICE_DATE','earliest_switch_date','2nd_line']]

In [19]:
patient_switch_therapy.head()

,PATIENT_ID,SERVICE_DATE,earliest_switch_date,2nd_line
0,325994833,2014-09-11,2014-05-22,2
1,105655736,2014-09-17,2015-01-28,2
2,832831479,2014-08-20,2016-01-05,2
3,832831479,2014-07-23,2016-01-05,2
4,832831479,2014-09-16,2016-01-05,2


In [20]:
RX_label = pd.concat([patient_one_drug,Patient_within_30_days],ignore_index=True)

RX_label = pd.concat([RX_label,patient_switch_therapy],ignore_index=True)

In [21]:
RX_label.head()

,2nd_line,PATIENT_ID,SERVICE_DATE,earliest_switch_date,latest_DIAG_Date
0,0,915900636,2019-12-31,NaT,2019-12-31
1,0,1167080703,2019-12-05,NaT,2019-12-05
2,0,1091256399,2019-10-17,NaT,2019-10-17
3,0,1727274390,2019-10-17,NaT,2019-11-18
4,0,76521234,2019-11-11,NaT,2019-12-09


In [22]:
RX_merged = RX_merged.merge(RX_label,how="left",on=['PATIENT_ID'])

RX_merged = RX_merged.rename(columns={'SERVICE_DATE_x':'SERVICE_DATE'}).drop(columns=['SERVICE_DATE_y'])

# Adding a default value
RX_merged['patient_date_2nd_line'] = 0

RX_merged['patient_date_2nd_line'] = np.where( ((RX_merged['2nd_line']==2) & (RX_merged['SERVICE_DATE']>=RX_merged['earliest_switch_date'])),1,0)

### Feature engineering

In [23]:
# Get the total qty for patient (per type of drug per day, except for other drug)
DRUG_QTY=pd.DataFrame(RX_merged[RX_merged['type']!='OTHER'].groupby(['PATIENT_ID','SERVICE_DATE','type'])['DSPNSD_QTY'].sum())
DRUG_QTY.reset_index(inplace=True)
DRUG_QTY = DRUG_QTY.rename(columns={"DSPNSD_QTY":"DRUG_QTY"})
DRUG_QTY = DRUG_QTY.drop_duplicates()

drug_qty = (DRUG_QTY.pivot_table(index=['PATIENT_ID','SERVICE_DATE'],columns='type', values='DRUG_QTY')
       .reset_index()
       .rename_axis(None, axis=1))
drug_qty = drug_qty.fillna(0)

In [24]:
# Get the days supply information (per drug per day, also except for other drug)
NDC_days = RX_merged[RX_merged['type']!='OTHER'].drop_duplicates(subset=['PATIENT_ID','SERVICE_DATE','NDC','DAYS_SUPPLY'])
days_supply = (NDC_days.pivot_table(index=['PATIENT_ID','SERVICE_DATE'],columns='NDC', values='DAYS_SUPPLY')
       .reset_index()
       .rename_axis(None, axis=1))
days_supply = days_supply.fillna(0)

In [25]:
# Rename dataframe columns before merging
days_supply.columns = days_supply.columns.map(str)
drug_qty.columns = drug_qty.columns.map(str)
days_supply.columns = 'days_' + days_supply.columns.values
drug_qty.columns = 'qty_' + drug_qty.columns.values

days_supply = days_supply.rename(columns={"days_PATIENT_ID":"PATIENT_ID", "days_SERVICE_DATE":"SERVICE_DATE"})
drug_qty = drug_qty.rename(columns={"qty_PATIENT_ID":"PATIENT_ID","qty_SERVICE_DATE":"SERVICE_DATE"})

In [26]:
# Deal with the refill code 
RX_merged['New_RX'] = np.where(RX_merged['REFILL_CODE']==0,1,0)
New_RX_count = pd.DataFrame(RX_merged.groupby(['PATIENT_ID','SERVICE_DATE'])['New_RX'].sum())
New_RX_count.reset_index(inplace=True)

In [27]:
New_RX_count.head()

,PATIENT_ID,SERVICE_DATE,New_RX
0,29425424,2015-06-22,52
1,29425424,2015-07-22,52
2,29425424,2015-08-24,52
3,29425424,2015-09-18,0
4,29425424,2015-10-13,0


In [28]:
# Deal with drug strength
RX_strength = RX_merged[RX_merged['type']!='OTHER'].drop_duplicates(subset=['PATIENT_ID','SERVICE_DATE','NDC'])


RX_strength['drug_strength'] = RX_strength['DRUG_STRENGTH'].str.extract(r'^(\d\.?\d+)', expand=False).astype(float)

DRUG_strength = (RX_strength.pivot_table(index=['PATIENT_ID','SERVICE_DATE'],columns='NDC', values='drug_strength')
       .reset_index()
       .rename_axis(None, axis=1))
DRUG_strength = DRUG_strength.fillna(0)

DRUG_strength.columns = DRUG_strength.columns.map(str)
DRUG_strength.columns = 'drug_strength_' + DRUG_strength.columns.values
DRUG_strength = DRUG_strength.rename(columns={"drug_strength_PATIENT_ID":"PATIENT_ID","drug_strength_SERVICE_DATE":"SERVICE_DATE"})

In [29]:
drug_qty.loc[:,'qty_AFI':'qty_VER'] = drug_qty.loc[:,'qty_AFI':'qty_VER'].astype(int)

In [33]:
days_supply.head()

,PATIENT_ID,SERVICE_DATE,days_2448354,days_2481554,days_2533754,days_2621654,days_9766304,days_54008013,days_54016413,days_54026913,days_54027121,days_54027223,days_69018721,days_69018821,days_69018921,days_78024915,days_78056651,days_78056751,days_78056761,days_78059451,days_78062051,days_78062061,days_78062851,days_78086001,days_78086714,days_78086742,days_78087421,days_78087463,days_78090961,days_78091661,days_78092361,days_93747306,days_93747489,days_93753656,days_93762056,days_310020130,days_310072010,days_310772010,days_378207105,days_378207193,days_378251191,days_378251278,days_378500193,days_378603493,days_591501902,days_603418016,days_781307912,days_781535631,days_832059530,days_904619546,days_13632012301,days_16571042103,days_16714046701,days_16714046801,days_16729003410,days_16729003415,days_16729003510,days_16729003515,days_16729003516,days_16729007212,days_16729007329,days_25021046274,days_42043018003,days_42291037490,days_47781010830,days_49884012591,days_49884012791,days_51991062010,days_51991062033,days_51991075910,days_51991075933,days_55111064730,days_59651018030,days_59651020508,days_59762285801,days_60505298503,days_60505325503,days_60505325508,days_60687011221,days_62559067030,days_62756025013,days_62756025083,days_62756051183,days_64980027606,days_64980027712,days_65162084306,days_65162084416,days_67457031105,days_67877017130,days_68001015504,days_68001015508,days_68084044821,days_68382020906,days_68382020910,days_68382038306,days_69097031602,days_69097094808,days_69097094903,days_70121146302,days_72205000660,days_72205000792,days_72485020460,days_72485020512,days_89141012301
0,29425424,2015-06-22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,29425424,2015-07-22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,29425424,2015-08-24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,29425424,2015-09-18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,29425424,2015-10-13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
DRUG_strength.loc[:,'drug_strength_2448354':] = DRUG_strength.loc[:,'drug_strength_2448354':].astype(int)

In [31]:
days_supply.loc[:,'days_2448354':] = days_supply.loc[:,'days_2448354':].astype(int)

In [34]:
RX_merged = RX_merged.drop(columns=['index','DSPNSD_QTY','DAYS_SUPPLY','DRUG_STRENGTH','REFILL_CODE'])

In [35]:
RX_merged = RX_merged.merge(New_RX_count,how="left",on=["PATIENT_ID",'SERVICE_DATE'])

In [36]:
RX_merged = RX_merged.merge(DRUG_strength,how="left",on=["PATIENT_ID",'SERVICE_DATE'])

In [37]:
RX_merged = RX_merged.merge(drug_qty,how="left",on=['PATIENT_ID','SERVICE_DATE'])

In [39]:
RX_merged = RX_merged.drop_duplicates(subset=['PATIENT_ID','SERVICE_DATE'])

In [42]:
RX_merged.shape

(213606, 96)

In [43]:
RX_final = RX_merged.merge(days_supply,how="left",on=['PATIENT_ID','SERVICE_DATE'])

In [44]:
RX_final.to_pickle("./RX_final.pkl")

In [61]:
second_line_patient = RX_final[(RX_final['2nd_line']==2) & (RX_final['SERVICE_DATE'] < RX_final['earliest_switch_date'])]

In [65]:
second_line_patient = second_line_patient.drop(columns=['NDC','type','Earliest_DIAG_Date','latest_DIAG_Date','New_RX_x'])

In [66]:
second_line_patient = second_line_patient.drop(columns=['SERVICE_DATE','earliest_switch_date','patient'])

,PATIENT_ID,SERVICE_DATE,Days_Diff,drug_type_AFI,drug_type_AI,drug_type_FAS,drug_type_IBR,drug_type_KIS,drug_type_LET,drug_type_OTHER,drug_type_TAM,drug_type_VER,first_drug,2nd_line,earliest_switch_date,patient_date_2nd_line,New_RX_y,drug_strength_2448354,drug_strength_2481554,drug_strength_2533754,drug_strength_2621654,drug_strength_9766304,drug_strength_54008013,drug_strength_54026913,drug_strength_54027121,drug_strength_54027223,drug_strength_69018721,drug_strength_69018821,drug_strength_69018921,drug_strength_78024915,drug_strength_78056751,drug_strength_78056761,drug_strength_78059451,drug_strength_78062051,drug_strength_78062061,drug_strength_93747306,drug_strength_93747489,drug_strength_93762056,drug_strength_310072010,drug_strength_310772010,drug_strength_378207105,drug_strength_378207193,drug_strength_378251191,drug_strength_378251278,drug_strength_378500193,drug_strength_591501902,drug_strength_603418016,drug_strength_781307912,drug_strength_832059530,drug_strength_13632012301,drug_strength_16714046701,drug_strength_16714046801,drug_strength_16729003410,drug_strength_16729003415,drug_strength_16729007212,drug_strength_16729007329,drug_strength_25021046274,drug_strength_42291037490,drug_strength_47781010830,drug_strength_49884012791,drug_strength_51991075910,drug_strength_51991075933,drug_strength_59651018030,drug_strength_59651020508,drug_strength_59762285801,drug_strength_60505325503,drug_strength_60505325508,drug_strength_62756051183,drug_strength_64980027606,drug_strength_64980027712,drug_strength_65162084306,drug_strength_65162084416,drug_strength_67457031105,drug_strength_68382038306,drug_strength_69097031602,drug_strength_69097094808,drug_strength_69097094903,drug_strength_70121146302,drug_strength_72205000660,drug_strength_72205000792,drug_strength_72485020460,drug_strength_72485020512,drug_strength_89141012301,qty_AFI,qty_AI,qty_FAS,qty_IBR,qty_KIS,qty_LET,qty_TAM,qty_VER,days_2448354,days_2481554,days_2533754,days_2621654,days_9766304,days_54008013,days_54016413,days_54026913,days_54027121,days_54027223,days_69018721,days_69018821,days_69018921,days_78024915,days_78056651,days_78056751,days_78056761,days_78059451,days_78062051,days_78062061,days_78062851,days_78086001,days_78086714,days_78086742,days_78087421,days_78087463,days_78090961,days_78091661,days_78092361,days_93747306,days_93747489,days_93753656,days_93762056,days_310020130,days_310072010,days_310772010,days_378207105,days_378207193,days_378251191,days_378251278,days_378500193,days_378603493,days_591501902,days_603418016,days_781307912,days_781535631,days_832059530,days_904619546,days_13632012301,days_16571042103,days_16714046701,days_16714046801,days_16729003410,days_16729003415,days_16729003510,days_16729003515,days_16729003516,days_16729007212,days_16729007329,days_25021046274,days_42043018003,days_42291037490,days_47781010830,days_49884012591,days_49884012791,days_51991062010,days_51991062033,days_51991075910,days_51991075933,days_55111064730,days_59651018030,days_59651020508,days_59762285801,days_60505298503,days_60505325503,days_60505325508,days_60687011221,days_62559067030,days_62756025013,days_62756025083,days_62756051183,days_64980027606,days_64980027712,days_65162084306,days_65162084416,days_67457031105,days_67877017130,days_68001015504,days_68001015508,days_68084044821,days_68382020906,days_68382020910,days_68382038306,days_69097031602,days_69097094808,days_69097094903,days_70121146302,days_72205000660,days_72205000792,days_72485020460,days_72485020512,days_89141012301
1,105655736,2014-09-17,0,1,0,0,0,0,0,0,0,0,AFI,2,2015-01-28,0,115,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [68]:
len(RX_final.PATIENT_ID.unique())

7052

In [62]:
first_drug_type_dummies =pd.get_dummies(second_line.first_drug,prefix='first_drug_type')

,PATIENT_ID,SERVICE_DATE,NDC,type,Earliest_DIAG_Date,Days_Diff,drug_type_AFI,drug_type_AI,drug_type_FAS,drug_type_IBR,drug_type_KIS,drug_type_LET,drug_type_OTHER,drug_type_TAM,drug_type_VER,first_drug,2nd_line,earliest_switch_date,latest_DIAG_Date,patient_date_2nd_line,New_RX_x,New_RX_y,drug_strength_2448354,drug_strength_2481554,drug_strength_2533754,drug_strength_2621654,drug_strength_9766304,drug_strength_54008013,drug_strength_54026913,drug_strength_54027121,drug_strength_54027223,drug_strength_69018721,drug_strength_69018821,drug_strength_69018921,drug_strength_78024915,drug_strength_78056751,drug_strength_78056761,drug_strength_78059451,drug_strength_78062051,drug_strength_78062061,drug_strength_93747306,drug_strength_93747489,drug_strength_93762056,drug_strength_310072010,drug_strength_310772010,drug_strength_378207105,drug_strength_378207193,drug_strength_378251191,drug_strength_378251278,drug_strength_378500193,drug_strength_591501902,drug_strength_603418016,drug_strength_781307912,drug_strength_832059530,drug_strength_13632012301,drug_strength_16714046701,drug_strength_16714046801,drug_strength_16729003410,drug_strength_16729003415,drug_strength_16729007212,drug_strength_16729007329,drug_strength_25021046274,drug_strength_42291037490,drug_strength_47781010830,drug_strength_49884012791,drug_strength_51991075910,drug_strength_51991075933,drug_strength_59651018030,drug_strength_59651020508,drug_strength_59762285801,drug_strength_60505325503,drug_strength_60505325508,drug_strength_62756051183,drug_strength_64980027606,drug_strength_64980027712,drug_strength_65162084306,drug_strength_65162084416,drug_strength_67457031105,drug_strength_68382038306,drug_strength_69097031602,drug_strength_69097094808,drug_strength_69097094903,drug_strength_70121146302,drug_strength_72205000660,drug_strength_72205000792,drug_strength_72485020460,drug_strength_72485020512,drug_strength_89141012301,qty_AFI,qty_AI,qty_FAS,qty_IBR,qty_KIS,qty_LET,qty_TAM,qty_VER,days_2448354,days_2481554,days_2533754,days_2621654,days_9766304,days_54008013,days_54016413,days_54026913,days_54027121,days_54027223,days_69018721,days_69018821,days_69018921,days_78024915,days_78056651,days_78056751,days_78056761,days_78059451,days_78062051,days_78062061,days_78062851,days_78086001,days_78086714,days_78086742,days_78087421,days_78087463,days_78090961,days_78091661,days_78092361,days_93747306,days_93747489,days_93753656,days_93762056,days_310020130,days_310072010,days_310772010,days_378207105,days_378207193,days_378251191,days_378251278,days_378500193,days_378603493,days_591501902,days_603418016,days_781307912,days_781535631,days_832059530,days_904619546,days_13632012301,days_16571042103,days_16714046701,days_16714046801,days_16729003410,days_16729003415,days_16729003510,days_16729003515,days_16729003516,days_16729007212,days_16729007329,days_25021046274,days_42043018003,days_42291037490,days_47781010830,days_49884012591,days_49884012791,days_51991062010,days_51991062033,days_51991075910,days_51991075933,days_55111064730,days_59651018030,days_59651020508,days_59762285801,days_60505298503,days_60505325503,days_60505325508,days_60687011221,days_62559067030,days_62756025013,days_62756025083,days_62756051183,days_64980027606,days_64980027712,days_65162084306,days_65162084416,days_67457031105,days_67877017130,days_68001015504,days_68001015508,days_68084044821,days_68382020906,days_68382020910,days_68382038306,days_69097031602,days_69097094808,days_69097094903,days_70121146302,days_72205000660,days_72205000792,days_72485020460,days_72485020512,days_89141012301
1,105655736,2014-09-17,78059451,AFI,2014-09-17,0,1,0,0,0,0,0,0,0,0,AFI,2,2015-01-28,NaT,0,1,115,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [ ]:
first_line_patient

In [ ]:
# RX_merged.to_pickle("./RX_final.pkl")

In [ ]:
first_line_patient = RX_final()